# EEG of mice and men

### Because of many trials, this code takes a lot of time to execute!

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from lfpykit.eegmegcalc import FourSphereVolumeConductor
from lfpykit.models import CurrentDipoleMoment, LineSourcePotential
import LFPy
from brainsignals.plotting_convention import mark_subplots, simplify_axes, cmap_v_e
from brainsignals import neural_simulations as ns

head_colors = ["#ffb380", "#74abff", "#b3b3b3", "#c87137"]
human_radii = np.array([89000., 90000., 95000., 100000.])  # (µm)
mice_radii = human_radii / 15
human_sigmas = [0.276, 1.65, 0.01, 0.465]  # (S/m)
mice_sigmas = human_sigmas

x = np.array([0])
y = np.array([0])

eps = 1e-2

z_human = np.array([human_radii[-1] - eps])
z_mice = np.array([mice_radii[-1] - eps])

r_elecs_human = np.array([x, y, z_human]).T # (µm)
r_elecs_mice = np.array([x, y, z_mice]).T # (µm)

human_4s = FourSphereVolumeConductor(r_elecs_human, human_radii, human_sigmas)
mice_4s = FourSphereVolumeConductor(r_elecs_mice, mice_radii, mice_sigmas)

# Cell morphologies downloaded from the Allen Brain Atlas
morph_names = {"L5_human_1": "H16.06.013.11.18.02_696537235_m.swc",
               "L5_human_2": "H16.06.010.01.03.05.02_599474744_m.swc",
               "L23_human_1": "H17.06.006.11.09.08_675799126_m.swc",
               "L23_human_2": "H16.06.012.11.08.04_668616935_m.swc",
               "L5_mice_1": "Tlx3-Cre_PL56_Ai14-338859.04.01.01_647469819_m.swc",
               "L5_mice_2": "Rbp4-Cre_KL100_Ai14-243058.04.02.01_553511048_m.swc",
               "L23_mice_1": "Slc17a6-IRES-Cre_Ai14-190263.04.01.01_706381804_m.swc",
               "L23_mice_2": "Cux2-CreERT2_Ai14-230637.05.01.01_661929017_m.swc",
              }

# URLs to where morphologies can be downloaded
morph_urls = {
    "H16.06.013.11.18.02_696537235_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/696537237',
    "H16.06.010.01.03.05.02_599474744_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/599474746',
    "H17.06.006.11.09.08_675799126_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/675799128',
    "H16.06.012.11.08.04_668616935_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/668616963',
    "Tlx3-Cre_PL56_Ai14-338859.04.01.01_647469819_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/647469823',
    "Rbp4-Cre_KL100_Ai14-243058.04.02.01_553511048_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/553511057',
    "Slc17a6-IRES-Cre_Ai14-190263.04.01.01_706381804_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/706381807',
    "Cux2-CreERT2_Ai14-230637.05.01.01_661929017_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/661929019',    
    }


def dipole_potential(elec_locs, dipole_pos, p):
    # Potential in infinite homogeneous medium
    r_ = elec_locs.T - dipole_pos
    V_e = 1000 * 1. / (4 * np.pi * human_sigmas[0]) * (np.dot(r_, p.T)
                    / np.linalg.norm(r_, axis=1) ** 3)
    return V_e

In [ ]:

dt = 2**-4
tstop = 20
np.random.seed(1234)
# Define synapse parameters
synapse_params = {
    'e' : 0.,                   # reversal potential
    'syntype' : 'Exp2Syn',       # synapse type
    'tau1' : 0.1,                 # synaptic time constant
    'tau2' : 1.,                 # synaptic time constant
    'weight' : 0.001,            # synaptic weight
    'record_current' : False,    # record synapse current
}

cell_params = {
        'passive': True,
        'passive_parameters': {"g_pas": 1 / 30000,
                               "e_pas": -70.},
        'nsegs_method': "lambda_f",
        'Ra': 150,
        'cm': 1.0,
        'lambda_f': 100,
        'dt': dt,
        'tstart': -1,
        'tstop': tstop,
        'v_init': -70,
        'pt3d': True,
        'custom_code': [os.path.join(ns.allen_folder, "remove_axon.hoc")]
    }

data_summary = {}
make_control_plots = False

for morph_name in morph_names.keys():
    
    morph_file = morph_names[morph_name]
    morph_path = os.path.join(ns.allen_folder, morph_file)
    
    if not os.path.isfile(morph_path):
        print(f"Downloading morphology {morph_file}")
        from urllib.request import urlopen
        import ssl
        u = urlopen(morph_urls[morph_file], context=ssl._create_unverified_context())
        localFile = open(morph_path, 'wb')
        localFile.write(u.read())
        localFile.close()

    cell_params["morphology"] = morph_path
    cell = LFPy.Cell(**cell_params)
    ns.align_cell_to_axes(cell)

    synidxs = cell.get_rand_idx_area_norm(nidx=100)
    
    cell.__del__()
    
    data_summary[morph_name] = {"idxs": synidxs,
                                "human_eeg": {"dp": np.zeros(len(synidxs)),
                                              "mp": np.zeros(len(synidxs))},
                                "mice_eeg": {"dp": np.zeros(len(synidxs)),
                                             "mp": np.zeros(len(synidxs))}
                               }

    for s_, synidx in enumerate(synidxs):
        print(morph_name, s_)
        cell = LFPy.Cell(**cell_params)
        ns.align_cell_to_axes(cell)

        synapse_params["idx"] = synidx
        synapse = LFPy.Synapse(cell, **synapse_params)
        synapse.set_spike_times(np.array([1.]))

        cell.simulate(rec_imem=True, rec_vmem=True)

        cdm = CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem

        max_time_idx = np.argmax(np.abs(cdm[2, :] - cdm[2, 0]))

        multi_dipoles, dipole_locs = cell.get_multi_current_dipole_moments(
            timepoints=np.array([max_time_idx]))

        somaloc = np.array([cell.x[0].mean(),
                            cell.y[0].mean(),
                            cell.z[0].mean(),])
        synloc = np.array([cell.x[synidx].mean(),
                            cell.y[synidx].mean(),
                            cell.z[synidx].mean()])

        dipole_loc = np.mean([somaloc, synloc], axis=0)
        
        dipole_loc_eeg_human = np.array([dipole_loc[0], 
                                         dipole_loc[1], 
                                         human_radii[0] - np.max(cell.z) - 100 + dipole_loc[2]])

        dipole_loc_eeg_mice = np.array([dipole_loc[0], 
                                        dipole_loc[1], 
                                        mice_radii[0] - np.max(cell.z) - 100 + dipole_loc[2]])

        M_human = human_4s.get_transformation_matrix(dipole_loc_eeg_human)
        M_mice = mice_4s.get_transformation_matrix(dipole_loc_eeg_mice)

        eeg_dp_human = 1000 * M_human @ cdm[:, max_time_idx] # (uV)
        eeg_dp_mice = 1000 * M_mice @ cdm[:, max_time_idx] # (uV)
        eeg_mp_human = 0
        eeg_mp_mice = 0

        for dp_idx in range(len(multi_dipoles)):

            dipole_loc_eeg_human = np.array([dipole_locs[dp_idx][0],
                                             dipole_locs[dp_idx][1], 
                                             human_radii[0] - np.max(cell.z) - 100 + dipole_locs[dp_idx][2]])
            dipole_loc_eeg_mice = np.array([dipole_locs[dp_idx][0],
                                            dipole_locs[dp_idx][1],
                                            mice_radii[0] - np.max(cell.z) - 100 + dipole_locs[dp_idx][2]])

            M_human = human_4s.get_transformation_matrix(dipole_loc_eeg_human)
            M_mice = mice_4s.get_transformation_matrix(dipole_loc_eeg_mice)

            eeg_mp_human += 1000 * M_human @ multi_dipoles[dp_idx][:, 0]# (uV)
            eeg_mp_mice += 1000 * M_mice @ multi_dipoles[dp_idx][:, 0]# (uV)

        data_summary[morph_name]["human_eeg"]["dp"][s_] = eeg_dp_human
        data_summary[morph_name]["human_eeg"]["mp"][s_] = eeg_mp_human
        
        data_summary[morph_name]["mice_eeg"]["dp"][s_] = eeg_dp_mice
        data_summary[morph_name]["mice_eeg"]["mp"][s_] = eeg_mp_mice
        
        human_error = np.abs((eeg_dp_human - eeg_mp_human) / eeg_mp_human) * 100
        mice_error = np.abs((eeg_dp_mice - eeg_mp_mice) / eeg_mp_mice) * 100
        
        if make_control_plots:
            # Making control plots
            x1 = np.max(np.abs(cell.x)) + 1000
            x0 = -x1
            dx = (x1 - x0) / 21

            z1 = np.max(cell.z) + 1000
            z0 = np.min(cell.z) - 1000
            dz = (z1 - z0) / 51

            grid_x, grid_z = np.mgrid[x0:x1:dx, z0:z1:dz]
            grid_y = np.zeros(grid_x.shape)

            # Define electrode parameters
            grid_elec_params = {
                'sigma': 0.3,      # extracellular conductivity
                'x': grid_x.flatten(),  # electrode requires 1d vector of positions
                'y': grid_y.flatten(),
                'z': grid_z.flatten()
            }

            grid_electrode = LFPy.RecExtElectrode(cell, **grid_elec_params)
            LFP = 1e6 * grid_electrode.get_transformation_matrix() @ cell.imem[:, max_time_idx]
            LFP_grid = LFP.reshape(grid_x.shape)

            r_elecs_grid = np.array([grid_x.flatten(), 
                                     grid_y.flatten(), 
                                     grid_z.flatten()])
            DP_grid = 1e3 * dipole_potential(r_elecs_grid, dipole_loc, 
                                       cdm[:, max_time_idx]).reshape(grid_x.shape)

            MP_grid = np.zeros(DP_grid.shape)
            for dp_idx in range(len(multi_dipoles)):
                MP_grid += 1e3 * dipole_potential(r_elecs_grid, dipole_locs[dp_idx], 
                                            multi_dipoles[dp_idx][:, 0]).reshape(grid_x.shape)

            print(morph_name, synidx, human_error, mice_error)

            vmax = np.ceil(np.max(np.abs(LFP_grid)))
            vmin = -vmax

            num = 21
            levels = np.logspace(-2.5, 0, num=num)
            scale_max = vmax#10**np.ceil(np.log10(np.max(np.abs(pot_homog))))
            levels_norm = scale_max * np.concatenate((-levels[::-1], levels))
            colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2)) for i in range(len(levels_norm) - 1)]
            colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)

            plt.close("all")
            fig = plt.figure(figsize=[6, 2])
            fig.subplots_adjust(left=0.05, right=0.97, bottom=0.2)

            ax_lfp_xz = fig.add_subplot(141, aspect=1,
                                    title="MC+VC", xlim=[x0, x1], ylim=[z0, z1], 
                                   frameon=False, xticks=[], yticks=[])

            ax_mp_xz = fig.add_subplot(142, aspect=1,
                                    title="multi-dipole", xlim=[x0, x1], ylim=[z0, z1], 
                                   frameon=False, xticks=[], yticks=[])

            ax_dp_xz = fig.add_subplot(143, aspect=1,
                                    title="single-dipole", xlim=[x0, x1], ylim=[z0, z1], 
                                   frameon=False, xticks=[], yticks=[])

            ax_dp_xz.text(0, z0 - 200, 
                          "Human 4S eeg: {:1.3f} pV\nMice 4S eeg: {:1.3f} pV".format(
                              eeg_mp_human[0] * 1e6, eeg_mp_mice[0] * 1e6), ha='center', va="top")

            ax_vm = fig.add_subplot(244, xlabel="time (ms)", ylabel="$V_{\rm m}$ (mV)")
            ax_cdm = fig.add_subplot(248, xlabel="time (ms)", ylabel="$P_z$ (nA µm)")

            ax_lfp_xz.plot(cell.x.T, cell.z.T, c='k')
            ax_lfp_xz.plot(cell.x[synidx].mean(), cell.z[synidx].mean(), 'bo', ms=2)
            ax_mp_xz.plot(cell.x.T, cell.z.T, c='k')
            ax_mp_xz.plot(cell.x[synidx].mean(), cell.z[synidx].mean(), 'bo', ms=2)
            ax_dp_xz.plot(cell.x.T, cell.z.T, c='k')
            ax_dp_xz.plot(cell.x[synidx].mean(), cell.z[synidx].mean(), 'bo', ms=2)

            ax_lfp_xz.contourf(grid_x, grid_z, LFP_grid, colors = colors_from_map,
                                           levels=levels_norm, extend='both')

            ax_mp_xz.contourf(grid_x, grid_z, MP_grid, colors = colors_from_map,
                                           levels=levels_norm, extend='both')

            img = ax_dp_xz.contourf(grid_x, grid_z, DP_grid, colors = colors_from_map,
                                           levels=levels_norm, extend='both')

            ax_vm.plot(cell.tvec, cell.vmem[0, :], 'k')
            ax_vm.plot(cell.tvec, cell.vmem[synidx, :], 'r')
            ax_vm.axvline(cell.tvec[max_time_idx], ls='--', c='gray')

            ax_cdm.plot(cell.tvec, cdm[2, :], 'k')
            ax_cdm.axvline(cell.tvec[max_time_idx], ls='--', c='gray')
            simplify_axes(fig.axes)

            cax = fig.add_axes([0.1, 0.15, 0.3, 0.01])
            cbar = plt.colorbar(img, cax=cax, orientation="horizontal")
            cbar.set_ticks([-vmax, -vmax/10, 0, vmax/10, vmax])

            figfolder = "allfigs"
            os.makedirs(figfolder, exist_ok=True)
            plt.savefig(os.path.join(figfolder, "cellsim_%s_%d.png" % (morph_name, synidx)), facecolor='w')

        cell.__del__()


In [ ]:
def plot_dipole_comparison(morph_name, synidx, ax_mp, ax_dp,
                           grid_elec_params, grid_x, grid_z):
    morph_file = morph_names[morph_name]
    cell_params["morphology"] = os.path.join(ns.allen_folder, morph_file)

    cell = LFPy.Cell(**cell_params)
    ns.align_cell_to_axes(cell)

    synapse_params["idx"] = synidx
    synapse = LFPy.Synapse(cell, **synapse_params)
    synapse.set_spike_times(np.array([1.]))

    cell.simulate(rec_imem=True, rec_vmem=True)
    cdm = CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem
    max_time_idx = np.argmax(np.abs(cdm[2, :] - cdm[2, 0]))

    multi_dipoles, dipole_locs = cell.get_multi_current_dipole_moments(
        timepoints=np.array([max_time_idx]))

    somaloc = np.array([cell.x[0].mean(),
                        cell.y[0].mean(),
                        cell.z[0].mean(),])
    synloc = np.array([cell.x[synidx].mean(),
                        cell.y[synidx].mean(),
                        cell.z[synidx].mean()])

    dipole_loc = np.mean([somaloc, synloc], axis=0)

    grid_electrode = LFPy.RecExtElectrode(cell, **grid_elec_params)
    LFP = 1e6 * grid_electrode.get_transformation_matrix() @ cell.imem[:, max_time_idx]
    LFP_grid = LFP.reshape(grid_x.shape)

    r_elecs_grid = np.array([grid_x.flatten(),
                             grid_y.flatten(),
                             grid_z.flatten()])
    DP_grid = 1e3 * dipole_potential(r_elecs_grid, dipole_loc,
                               cdm[:, max_time_idx]).reshape(grid_x.shape)

    MP_grid = np.zeros(DP_grid.shape)
    for dp_idx in range(len(multi_dipoles)):
        MP_grid += 1e3 * dipole_potential(r_elecs_grid, dipole_locs[dp_idx],
                                    multi_dipoles[dp_idx][:, 0]).reshape(grid_x.shape)

    vmax = 5#np.ceil(np.max(np.abs(LFP_grid)))
    vmin = -vmax

    #print(vmax)
    num = 11
    levels = np.logspace(-1.5, 0, num=num)
    scale_max = vmax#10**np.ceil(np.log10(np.max(np.abs(pot_homog))))
    levels_norm = scale_max * np.concatenate((-levels[::-1], levels))
    colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2)) for i in range(len(levels_norm) - 1)]
    colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)

    ax_mp.plot(cell.x.T, cell.z.T, c='k', lw=1)
    ax_mp.plot(cell.x[synidx].mean(), cell.z[synidx].mean(), 'bo', ms=2)
    ax_dp.plot(cell.x.T, cell.z.T, c='k', lw=1)
    ax_dp.plot(cell.x[synidx].mean(), cell.z[synidx].mean(), 'bo', ms=2)

    ax_mp.contourf(grid_x, grid_z, MP_grid, colors=colors_from_map,
                                   levels=levels_norm, extend='both')

    ax_dp.contourf(grid_x, grid_z, DP_grid, colors=colors_from_map,
                                   levels=levels_norm, extend='both')

    simplify_axes(fig.axes)

    del cell
    del synapse
    del grid_electrode


In [ ]:
plt.close("all")
fig = plt.figure(figsize=[6, 6.])
fig.subplots_adjust(wspace=0.55, bottom=0.05, top=0.88, right=0.99, left=0.0)

ax1 = fig.add_axes([0.01, 0.65, 0.3, 0.35], aspect=1, xticks=[], yticks=[], frameon=False,
                   xlim=[-1.2e5, 1.2e5], ylim=[-1.2e5, 1.2e5])
for i in range(4):
    ax1.add_patch(plt.Circle((0, 0), radius=human_radii[-1 - i],
                                   color=head_colors[-1-i],
                                   fill=True, ec='none', lw=.1))

    ax1.add_patch(plt.Circle((0.9e5, 0.7e5), radius=mice_radii[-1 - i],
                                   color=head_colors[-1-i],
                                   fill=True, ec='none', lw=.0))

ax1.text(0, 0.7e5, "human", ha="center")
ax1.text(0.8e5, 0.8e5, "mouse")
ax1.plot([0.5e5, 0.6e5], [0.5e5, 0.5e5], lw=1, c='k')
ax1.text(0.55e5, 0.47e5, "1 cm", va="top", ha="center")
ax2 = fig.add_axes([0.3, 0.65, 0.68, 0.35], aspect=1,
                   xticks=[], yticks=[], frameon=False)

x_shift = 0
cell_id_mice = 0
cell_id_human = 0
for morph_name in morph_names.keys():
    morph_file = morph_names[morph_name]
    cell_params["morphology"] = os.path.join(ns.allen_folder, morph_file)

    cell = LFPy.Cell(**cell_params)
    ns.align_cell_to_axes(cell)

    ax2.plot(cell.x.T + x_shift, cell.z.T, c='k')
    if "mice" in morph_name:
        preletter = "M"
        cell_id_mice += 1
        cell_id = cell_id_mice
    elif "human" in morph_name:
        preletter = "H"
        cell_id_human += 1
        cell_id = cell_id_human
    else:
        raise RuntimeError("unrecognized name")
    ax2.text(x_shift, -550, "%s%d" % (preletter, cell_id), ha='center', color='r')

    x_shift += np.max(cell.x) + 450
    cell.__del__()

    #cell_id += 1
ax2.plot([2500, 2500], [1500, 500], lw=1, c='k')
ax2.text(2550, 1000, "1 mm", va="center", ha='left')
mark_subplots(ax1, "A", ypos=1.0, xpos=0.05)
mark_subplots(ax2, "B", ypos=1.03, xpos=0.05)

axtitles = {"mice_eeg": "mouse head",
            "human_eeg": "human head",}

for s, species in enumerate(["mice_eeg", "human_eeg", ]):
    i = 0
    case_names = []
    ax_eeg = fig.add_subplot(3, 4, 7 + s, yscale="log", title=axtitles[species],
                             ylabel="EEG amplitude (nV)", ylim=[1e-4, 1e0])
    ax_err = fig.add_subplot(3, 4, 11 + s, yscale="log", title=axtitles[species],
                             ylabel="relative error", ylim=[1e-4, 1e0])
    species_name = species.split("_")[0]
    print(species_name)
    for morph_name in morph_names.keys():
        if not species_name in morph_name:
            continue
        data_ = data_summary[morph_name][species]
        error = np.abs((data_["dp"] - data_["mp"]) / np.max(np.abs(data_["mp"])))
        eeg_ = np.abs(data_["mp"])
        x_ = np.zeros(len(eeg_)) + i

        ax_eeg.scatter(x_, eeg_ * 1000, s=2, c='0.9')
        ax_err.scatter(x_, error, s=2, c='0.9')
        ax_err.plot(i, np.mean(error), 'o', c='k', ms=4)
        ax_eeg.plot(i, np.mean(eeg_ * 1000), 'o', c='k', ms=4)
        case_names.append(morph_name)

        i += 1
    preletter = species_name[0].capitalize()
    ax_eeg.set_xticks(np.arange(i))
    ax_eeg.set_xticklabels(["%s%d" % (preletter, i_) for i_ in np.arange(1, i+1)])
    ax_err.set_xticks(np.arange(i))
    ax_err.set_xticklabels(["%s%d" % (preletter, i_) for i_ in np.arange(1, i+1)])

mark_subplots(fig.axes[-4:], ["D", "F", "E", "G"])
simplify_axes(fig.axes)

dipole_examples = [["L5_mice_2", 113],
                   ["L23_human_1", 634],
                   ["L5_human_1", 209],
                  ]
# Making control plots
x1 = 1350
x0 = -1200
dx = (x1 - x0) / 21

z1 = 1900
z0 = -1800
dz = (z1 - z0) / 51

grid_x, grid_z = np.mgrid[x0:x1:dx, z0:z1:dz]
grid_y = np.zeros(grid_x.shape)

# Define electrode parameters
grid_elec_params = {
    'sigma': 0.3,      # extracellular conductivity
    'x': grid_x.flatten(),  # electrode requires 1d vector of positions
    'y': grid_y.flatten(),
    'z': grid_z.flatten()
}

for i, (morph_name, synidx) in enumerate(dipole_examples):
    ax_mp = fig.add_axes([0.05, 0.42 - i * 0.24, 0.17, 0.24], aspect=1,
                            xlim=[x0, x1], ylim=[z0, z1],
                           frameon=False, xticks=[], yticks=[])

    ax_dp = fig.add_axes([0.22, 0.42 - i * 0.24, 0.17, 0.24], aspect=1,
                            xlim=[x0, x1], ylim=[z0, z1],
                           frameon=False, xticks=[], yticks=[])

    if i == 0:
        mark_subplots(ax_mp, "C", ypos=1.05, xpos=0.05)
        ax_mp.set_title("MC+VC", y=0.9)
        ax_dp.set_title("dipole approximation", y=0.9)

    plot_dipole_comparison(morph_name, synidx, ax_mp, ax_dp, grid_elec_params, grid_x, grid_z)

plt.savefig("of_mice_and_men.pdf")
